In [1]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil

import flopy
from modflowapi import ModflowApi

from bmi.wrapper import BMIWrapper

from liss_settings import libmf6, get_dflow_grid_name, get_dflow_dtuser, get_modflow_coupling_tag

C:\Users\jdhughes\miniconda3\envs\liss\lib\site-packages\bmi\__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
C:\Users\jdhughes\miniconda3\envs\liss\lib\site-packages\pkg_resources\__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
C:\Users\jdhughes\miniconda3\envs\liss\lib\site-packages\pkg_resources\__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap.responses')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-

In [2]:
control_path = pl.Path("../dflow-fm/coarse/tides/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
grid_name = get_dflow_grid_name(control_path)
print(grid_name)

LIS_modflow_bathy2_net


In [3]:
dflowfm_dtuser = get_dflow_dtuser(control_path)
print(dflowfm_dtuser)

300.0


#### Set unit conversion factors

In [4]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((3.28082**3) * 86400.)

#### Set the MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [5]:
mf_couple_freq_hours = 0.25  #Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
dflow_per_mf = int(mf_couple_freq / dflowfm_dtuser)

In [6]:
print(f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\nMODFLOW coupled to D-FLOW FM every {dflow_per_mf} output time step ({dflowfm_dtuser} sec.)") 

MODFLOW coupling frequency 0.25 hours
MODFLOW coupled to D-FLOW FM every 3.0 output time step (300.0 sec.)


In [7]:
mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

MODFLOW coupling tag: 15.00M


In [8]:
nstp = int(86400.0 / (dflow_per_mf * dflowfm_dtuser))
print(f"MODFLOW time steps per day: {nstp}")

MODFLOW time steps per day: 96


#### Set a few variables for controlling coupling

In [9]:
HDRY = -1e30
DEPTH_MIN = 0.1

#### Print the path of the modflow6 shared library

In [10]:
str(libmf6), libmf6.is_file()

('C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Scripts\\libmf6.dll', True)

#### Load the D-FLOW to MODFLOW weights

In [11]:
npzfile = np.load(f"../mapping/dflow2mfghb_{grid_name}.npz")
npzfile

NpzFile '../mapping/dflow2mfghb_LIS_modflow_bathy2_net.npz' with keys: dflow2mfghb, ghbmask, ghb2qext

In [12]:
dflow2mfghb = npzfile["dflow2mfghb"]
ghbmask = npzfile["ghbmask"]
ghb2qext = npzfile["ghb2qext"]

#### Define paths for the model simulation

In [13]:
mf_base_path = pl.Path("../modflow/greenport500ft/base/").resolve()
mf_run_path = pl.Path(f"../modflow/greenport500ft/run_{mf_tag}/").resolve()

#### Load the base MODFLOW model

In [14]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_base_path)
gwf = sim.get_model()

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package obs...
    loading package sto...
    loading package oc...
    loading package ghb...
    loading package wel...
    loading package drn...
    loading package rch...
  loading solution package modflow...


In [15]:
sim.set_sim_path(mf_run_path)

#### Change TDIS to hourly time steps

In [16]:
tdis = sim.get_package("TDIS")
perioddata = tdis.perioddata.array

In [17]:
perioddata["nstp"] = nstp
tdis.perioddata = perioddata

#### Write the new model files

In [18]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package modflow...
  writing model modflow...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package obs-1...
    writing package sto...
    writing package oc...
    writing package ghb-1...
    writing package wel-1...
    writing package drn_obs...
    writing package drn-1...
    writing package rch-1...


#### Define base GHB variables

In [19]:
ghb_data0 = gwf.ghb.stress_period_data.get_dataframe()[0]
ghb_data0

,layer,row,column,bhead,cond,iface,boundname
0,0,26,0,0.0,24999.999994,0,'sound'
1,0,27,0,0.0,25000.000000,0,'sound'
2,0,28,0,0.0,24999.999994,0,'sound'
3,0,29,0,0.0,24231.956659,0,'sound'
4,0,25,1,0.0,25000.000006,0,'sound'
...,...,...,...,...,...,...,...
730,0,14,56,0.0,22494.219708,0,'peconic'
731,0,15,56,0.0,25000.000006,0,'peconic'
732,0,16,56,0.0,24999.999994,0,'peconic'
733,0,17,56,0.0,25000.000000,0,'peconic'


#### Setup and initialize D-FLOW FM

You will need to set `dflow_dirpath` to the correct directory on your machine.

In [20]:
# dflow_dirpath = os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin")
# dflow_deps_dirpath = (
#     os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\share\bin"),
# )
dflow_dirpath = pl.Path(r"X:\Work\new_york\dflow-fm\dflowfm_dll") 
dflow_base = pl.Path(r"X:\Work\new_york\dflow-fm\coarse\tides\base")
dflow_working = pl.Path(r"X:\Work\new_york\dflow-fm\coarse\tides\run")
dflow_config = dflow_working / "FlowFM.mdu"

In [21]:
if dflow_working.is_dir():
    shutil.rmtree(dflow_working)
shutil.copytree(dflow_base, dflow_working)

WindowsPath('X:/Work/new_york/dflow-fm/coarse/tides/run')

In [22]:
os.environ["PATH"]

'C:\\Users\\jdhughes\\miniconda3\\envs\\liss;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Library\\mingw-w64\\bin;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Library\\usr\\bin;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Library\\bin;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Scripts;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\bin;C:\\Users\\jdhughes\\miniconda3\\condabin;C:\\Program Files\\Parallels\\Parallels Tools\\Applications;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\bin;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\bin\\release;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\libfabric\\bin;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\libfabric\\bin\\utils;C:\\Program Files (x86)\\Intel\\oneAPI\\compiler\\latest\\windows\\redist\\intel64_win\\compiler;C:\\Program Files (x86)\\Intel\\oneAPI\\compiler\\latest\\windows\\redist\\ia32_win\\compiler;C:\\Program Files (x86)\\Intel\\oneAPI\\tbb\\latest\\redist\\intel64\\vc_mt;C:\\Program Files (x86)\\Int

In [23]:
# Add dflowfm dll folder to PATH so that it can be found by the BMIWrapper
os.environ["PATH"] = (
    str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
)

In [24]:
os.environ["PATH"]

'X:\\Work\\new_york\\dflow-fm\\dflowfm_dll;C:\\Users\\jdhughes\\miniconda3\\envs\\liss;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Library\\mingw-w64\\bin;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Library\\usr\\bin;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Library\\bin;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\Scripts;C:\\Users\\jdhughes\\miniconda3\\envs\\liss\\bin;C:\\Users\\jdhughes\\miniconda3\\condabin;C:\\Program Files\\Parallels\\Parallels Tools\\Applications;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\bin;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\bin\\release;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\libfabric\\bin;C:\\Program Files (x86)\\Intel\\oneAPI\\mpi\\latest\\libfabric\\bin\\utils;C:\\Program Files (x86)\\Intel\\oneAPI\\compiler\\latest\\windows\\redist\\intel64_win\\compiler;C:\\Program Files (x86)\\Intel\\oneAPI\\compiler\\latest\\windows\\redist\\ia32_win\\compiler;C:\\Program Files (x86)\\Intel\\oneAPI\\tbb\\latest\\redist\\i

In [25]:
(pl.Path(dflow_dirpath) / "dflowfm.dll").is_file()

True

#### Initialize D-Flow FM

In [26]:
dflowfm = BMIWrapper(
    engine="dflowfm",
    configfile=os.path.abspath(dflow_config),
)

In [27]:
dflowfm.initialize()

#### Get data from D-FLOW FM

In [28]:
ndxi = int(dflowfm.get_var("ndxi"))
ndx = int(dflowfm.get_var("ndx"))
x = dflowfm.get_var("xz")
y = dflowfm.get_var("yz")
z = dflowfm.get_var("bl")
xy = [(xx, yy) for (xx, yy) in zip(x, y)]
ndx, ndxi, x.shape, y.shape

(8412, 8323, (8412,), (8412,), 300.0)

In [29]:
dflowfm.get_var("s1")

array([0., 0., 0., ..., 0., 0., 0.])

In [30]:
v = dflowfm.get_var("hs")
v.shape, v

((8412,),
 array([26.42575318, 24.9297905 , 23.53244183, ..., 18.71800344,
        20.31243107, 21.52212007]))

In [31]:
qext = np.zeros(ndx)
qext.shape, qext

((8412,), array([0., 0., 0., ..., 0., 0., 0.]))

In [32]:
# dflowfm.set_var("qext", qext)

In [33]:
qext_cum = np.zeros(ndx)
qext_cum.shape

(8412,)

In [34]:
vextcum = dflowfm.get_var("vextcum")
vextcum.shape, vextcum

((8412,), array([0., 0., 0., ..., 0., 0., 0.]))

#### Initialize MODFLOW using MODFLOW API

In [35]:
mf6 = ModflowApi(str(libmf6), working_directory=mf_run_path)

In [36]:
mf6.initialize()

#### Define MODFLOW variable tags and set pointer to MODFLOW variables

In [37]:
ghb_bhead_tag = mf6.get_var_address("BHEAD", "MODFLOW", "GHB-1")
ghb_cond_tag = mf6.get_var_address("COND", "MODFLOW", "GHB-1")
ghb_flow_tag = mf6.get_var_address("SIMVALS", "MODFLOW", "GHB-1")

In [38]:
ghb_bhead_ptr = mf6.get_value_ptr(ghb_bhead_tag)
ghb_cond_ptr = mf6.get_value_ptr(ghb_cond_tag)
ghb_flow = np.zeros(ghb_bhead_ptr.shape)

#### Create dictionaries for saving modified GHB data

In [39]:
elev_dict = {}
cond_dict = {}
qext_dict = {}

#### Function to update MODFLOW GHB data

In [40]:
def update_mf(key, s, d):
    mask = d == 0.0
    s[mask] = 0.0
    mult = np.full(d.shape, 1.0)
    mult[mask] = 0.0

    ghb_head = ghb_data0["bhead"].to_numpy()
    ghb_head[ghbmask] = dflow2mfghb.dot(s)[ghbmask] * m2ft
    ghb_cond = ghb_data0["cond"].to_numpy()
    ghb_cond[ghbmask] = ghb_cond[ghbmask] * dflow2mfghb.dot(mult)[ghbmask]
    
    ghb_bhead_ptr[:] = ghb_head[:]
    ghb_cond_ptr[:] = ghb_cond[:]

    # update results dictionary
    elev_dict[key] = ghb_head.copy()
    cond_dict[key] = ghb_cond.copy()

#### Function to update D-Flow FM Qext data

In [41]:
def update_dflow(key, d):
    ghb_flow = -mf6.get_value(ghb_flow_tag) * cfd2cms
    dflow_qext = ghb2qext.dot(ghb_flow)
    dflow_qext[d == 0.0] = 0.0
    qext_cum[:ndxi] += dflow_qext[:ndxi]

    qext[:ndxi] = dflow_qext[:ndxi]
    dflowfm.set_var("qext", qext)

    # update results dictionaries
    qext_dict[key] = qext[:ndxi].copy()


#### Run each time step

In [42]:
print(
    f"DFLOWFM current_time: {dflowfm.get_current_time():15,.1f} sec. ({dflowfm.get_current_time()/86400.:15,.1f} days)\n"
     + f"DFLOWFM end_time:     {dflowfm.get_end_time():15,.1f} sec. ({dflowfm.get_end_time()/86400.:15,.1f} days)"
)

DFLOWFM current_time:             0.0 sec. (            0.0 days)
DFLOWFM end_time:         7,776,000.0 sec. (           90.0 days)


In [43]:
idx = 0
t0 = time.perf_counter()
current_time = dflowfm.get_current_time()
end_time = dflowfm.get_end_time()
while current_time <= end_time:
    ontime = dflowfm.get_current_time()
    onday = ontime / 86400.
    frac_comp = current_time / end_time
    print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
    dflowfm.update()

    current_time = dflowfm.get_current_time()
    if current_time % mf_couple_freq == 0.0:
        print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d}) ***", end="\r")
        s = dflowfm.get_var("s1")[:ndxi]
        d = dflowfm.get_var("hs")[:ndxi]

        mf6.prepare_time_step(mf6.get_time_step())
        update_mf(str(idx), s, d)
        mf6.do_time_step()
        mf6.finalize_time_step()
        update_dflow(str(idx), d)

        idx += 1

    
    if current_time == end_time:
        break

vextcum = dflowfm.get_var("vextcum")

t1 = time.perf_counter()
print(f"\nrun time: {(t1 - t0) / 60.} min")

Current time:     7,776,000.0 (    89.997 days) - 100.00% complete - (8639) ***
run time: 81.27547328333337 min


In [44]:
# Finalize
mf6.finalize()

In [45]:
dflowfm.finalize()

#### Save ghb elevation and conductance data to compressed files

In [46]:
np.savez_compressed(f"{mf_run_path}/ghb_elev.npz", **elev_dict)
np.savez_compressed(f"{mf_run_path}/ghb_cond.npz", **cond_dict)
np.savez_compressed(f"{mf_run_path}/qext.npz", **qext_dict)